## Testing the fast class of polynomials for correctness

We verify that our class Poly works correctly by mimicking the computation in SageMath.

In [1]:
load("poly_class.sage")
import time

In [2]:
load("poly_class.sage")
class TestPoly():
    def __init__(self):
        self.test_setup()
    
    def test_setup(self):
        self.N = 2 ** 4
        self.Q = 2 ** 3
        self.R = PolynomialRing(ZZ, 'x')
        self.x = self.R.gen()
        self.quo = self.x ** self.N + 1
        self.R = PolynomialRing(ZZ, 'x').quotient(self.quo)
        
        self.RQ = PolynomialRing(Zmod(self.Q), 'x').quotient(self.quo)
        Poly.setup(self.N, self.Q)
        
    def run_test_add_mod(self):
        a = self.RQ.random_element()
        b = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q)
        b_poly = Poly(b.list(), self.Q)
        sum = a + b
        sum_poly = a_poly + b_poly
        assert sum == self.RQ(sum_poly.list())
        
    def run_test_sub(self):
        a = self.R.random_element()
        b = self.R.random_element()
        a_poly = Poly(a.list(), 0)
        b_poly = Poly(b.list(), 0)
        diff = a - b
        diff_poly = a_poly - b_poly
        assert diff == self.R(diff_poly.list())
        
    def run_test_mul_mod(self):
        a = self.RQ.random_element()
        b = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q)
        b_poly = Poly(b.list(), self.Q)
        prod = a * b
        prod_poly = a_poly * b_poly
        assert prod == self.RQ(prod_poly.list())
        
    def run_test_reduce(self):
        a = self.RQ.random_element()
        a_poly = Poly(a.list(), 0)
        a_poly = a_poly % (self.Q//2 - 1)
        a = [ZZ(i) % (self.Q//2 - 1) for i in a.list()]
        assert self.R(a) == self.R(a_poly.list())
        
    def run_test_mod_switch(self):
        a = self.RQ.random_element()
        b = self.RQ.random_element()
        a_poly = Poly(a.list(), self.Q) % (self.Q//2 - 1)
        b_poly = Poly(b.list(), self.Q) % (self.Q//2 - 1)
        c_poly = (a_poly * b_poly) % self.Q
        a = [Zmod(self.Q//2 - 1)(i) for i in a.list()]
        b = [Zmod(self.Q//2 - 1)(i) for i in b.list()]
        ring = PolynomialRing(Zmod(self.Q//2 - 1), 'x').quotient(self.quo)
        a = ring(a)
        b = ring(b)
        c = [ZZ(i) % self.Q for i in (a * b)]
        assert self.R(c) == self.R(c_poly.list())
        

In [3]:
Test = TestPoly()
Test.run_test_add_mod()
Test.run_test_sub()
Test.run_test_mul_mod()
Test.run_test_reduce()
Test.run_test_mod_switch()

In [3]:
import cProfile
load("poly_class.sage")

class Runtimes:
    @classmethod
    def parameters(cls, N=2**15, Q=2**1000, iter=5):
        cls.N = N
        cls.Q = Q
        assert iter > 0
        cls.i = iter
        Poly.setup(cls.N, cls.Q)
        
    def __init__(self):
        self.parameters()
        
    def random(self):
        self.a = Poly.random()
        self.b = Poly.random()
        
    def run(self):
            
        print(f"Running tests with N={self.N}, log(Q)={log(self.Q,2)}, and {self.i} iterations")
        print(f"Preparing...")
        self.random()
        t = time.time()
        self.array = [Poly.random() for _ in range(self.i)]
        print(f"Time for random polynomials: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        ## ARITHMETIC
    
        t = time.time()
        _ = [self.a + self.array[_] for _ in range(self.i)]
        print(f"Time for addition: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        t = time.time()
        _ = [self.a * self.array[_] for _ in range(self.i)]
        print(f"Time for multiplication: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        t = time.time()
        _ = [self.array[_] / (self.Q//2-1) for _ in range(self.i)]
        print(f"Time for scalar division: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        t = time.time()
        _ = [self.array[_] % (self.Q//2-1) for _ in range(self.i)]
        print(f"Time for modular reduction: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        t = time.time()
        _ = [self.array[_].rescale(self.Q//2-1) for _ in range(self.i)]
        print(f"Time for rescaling: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        ## AUTOMORPHISMS
        
        t = time.time()
        _ = [self.array[_].auto5() for _ in range(self.i)]
        print(f"Time for auto5: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        t = time.time()
        _ = [self.array[_].auto_inverse() for _ in range(self.i)]
        print(f"Time for auto_inverse: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        t = time.time()
        _ = [self.array[_].auto(_) for _ in range(self.i)]
        print(f"Time for auto: {RealField(20)(1000 * (time.time() - t) / self.i)} ms ")
        
        
        
        
        

# profiler = cProfile.Profile()
# profiler.enable()

Run = Runtimes()
Run.run()

# profiler.disable()
# profiler.print_stats(sort='time')
    

Running tests with N=32768, log(Q)=1000, and 5 iterations
Preparing...
Time for random polynomials: 333.11 ms 
Time for addition: 1.0946 ms 
Time for multiplication: 93.376 ms 
Time for scalar division: 2.6870 ms 
Time for modular reduction: 1.7570 ms 
Time for rescaling: 2.0564 ms 
Time for auto5: 11.598 ms 
Time for auto_inverse: 2.4470 ms 
Time for auto: 11.627 ms 


TypeError: Poly.auto() missing 1 required positional argument: 'index'